# ONNXRUNTIME

## Setup ONNXRUNTIME

In [1]:
%pip install torch torchvision
%pip install onnx onnxruntime

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 6.5 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib
import numpy as np
import torch.onnx
import onnx
import onnxruntime

## Train Model

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)  
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

MODEL_DIR = pathlib.Path("./models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(model.state_dict(), MODEL_DIR / "original_model.p")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.431502
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.335230
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.070259
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.854500
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.049398
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.075935
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.449540
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.645479
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.750051
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.377498
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.418585
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.252689
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.238688
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.174919
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.075476
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.406093
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.272889
Train Epoch: 1 [544/60000 (1%)]	Loss: 0.944239
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.247047
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.294861
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.147572
Train Epoch: 1 [67

## Export to ONNX


In [9]:
x, _ = next(iter(train_loader))
torch.onnx.export(model,              
                  x,                         
                  MODEL_DIR / "mnist_model.onnx",  
                  export_params=True,        
                  opset_version=10,          
                  do_constant_folding=True,  
                  input_names = ['input'],   
                  output_names = ['output'], 
                  dynamic_axes={'input' : {0 : 'batch_size'},    
                                'output' : {0 : 'batch_size'}})

## Compare size

In [10]:
%ls -lh {MODEL_DIR}

total 192K
-rw-rw-r-- 1 harzad harzad 82K Dis  20 07:29 mnist_model.onnx
-rw-rw-r-- 1 harzad harzad 82K Dis  20 07:25 original_model.p
-rw-rw-r-- 1 harzad harzad 23K Dis  19 22:24 quantized_model.p


## Run inference and test simalirity

In [13]:
torch_out = model(x)

onnx_model = onnx.load(MODEL_DIR / "mnist_model.onnx")
onnx.checker.check_model(onnx_model)

ort_session = onnxruntime.InferenceSession(MODEL_DIR / "mnist_model.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


## Quantization